# 0. PROPUESTA JULIO DÍAZ




# 1. PARTE 1 CARGA DE DATOS


In [14]:
import pandas as pd


path = 'pe_properties.csv'
df=pd.read_csv(path)





In [15]:
df

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,1qOI7X9aqAcO7PzjX1B29Q==,Propiedad,2021-06-03,2021-08-27,2021-06-03,NaN,NaN,Perú,Lima,Lima,...,5.0,NaN,NaN,450000.0,USD,NaN,CASA DE PLAYA VENDO EN LAS LAGUNAS,<b>CASA DE PLAYA VENDO EN LAS LAGUNAS</b><br><...,Casa,Venta
1,J350rcSI7Fx+92Wb1ev7rw==,Propiedad,2021-06-03,2021-07-07,2021-06-03,NaN,NaN,Perú,Arequipa,Arequipa,...,6.0,NaN,NaN,1050000.0,USD,NaN,HERMOSA CASA DE ENSUEÑOS EN CERRO COLORADO,<b>HERMOSA CASA DE ENSUEÑOS EN CERRO COLORADO<...,Casa,Venta
2,P+3USxoY2w5HQwg1N8JCtg==,Propiedad,2021-06-03,9999-12-31,2021-06-03,-8.095431,-79.053100,Perú,La Libertad,Trujillo,...,4.0,NaN,NaN,179000.0,USD,NaN,[ VENTA ]: ESTRENA TU CASA EN URB. SOL Y LUZ C...,<b>[ VENTA ]: ESTRENA TU CASA EN URB. SOL Y LU...,Casa,Venta
3,I9llUYb61GVgLNDUJ6CT5A==,Propiedad,2021-06-03,2021-06-04,2021-06-03,-13.840915,-76.250830,Perú,Ica,Ica,...,NaN,1120.0,NaN,52000.0,USD,Mensual,Lote venta,Hábitat VIp vende exclusivo terreno 1120 m2 e...,Lote,Venta
4,d7Sw3mL1Vq0dmzoXNpJUNQ==,Propiedad,2021-06-03,2021-06-04,2021-06-03,-12.799091,-76.569120,Perú,Lima,Lima,...,NaN,500.0,NaN,70000.0,USD,Mensual,Lote venta,Hábitat Vip Vende lote 500 m2 \r\nUbicado a ta...,Lote,Venta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100619,WyZKSpo/YvheNozTnPoVHA==,Propiedad,2021-05-27,2021-06-15,2021-05-27,NaN,NaN,Perú,Arequipa,Arequipa,...,5.0,NaN,NaN,428000.0,USD,NaN,CASA EN VENTA EN EL CERCADO DE AREQUIPA,<b>CASA EN VENTA EN EL CERCADO DE AREQUIPA</b>...,Casa,Venta
100620,6cy+KTI2+ctSeXDIycRZ9Q==,Propiedad,2021-05-27,2021-10-04,2021-05-27,-12.260809,-76.884048,Perú,Lima,Lima,...,5.0,NaN,NaN,699000.0,USD,NaN,Venta de Exclusiva Casa en Condominio Paso Chico,<b>Venta de Exclusiva Casa en Condominio Paso ...,Casa,Venta
100621,yatXnRoWUVmgvJ16zXUY3Q==,Propiedad,2021-05-27,2021-08-16,2021-05-27,-12.094668,-76.899478,Perú,Lima,Lima,...,6.0,350.0,288.0,300000.0,USD,Mensual,VENDO 2 lindas casas en un mismo terreno en La...,"VENDO 2 LINDAS CASAS EN UN MISMO TERRENO, LOS ...",Casa,Venta
100622,j2m055N7hCGWkRcgMvTk2w==,Propiedad,2021-05-27,9999-12-31,2021-05-27,-10.742903,-77.753817,Perú,Lima,Barranca,...,6.0,250.0,1686.0,758497.0,USD,Mensual,OPORTUNIDAD LOCAL EN VENTA PARA TO TIPO DE INV...,Local Ubicado en la urb virgen de chapi en Bar...,Departamento,Venta


In [16]:
### PASO 1 DIVIR ENTRE /1000 EL PRECIO
### REVISION DE NANS
nul=df.isnull().sum()

df = df[df['price'].notna()]

In [17]:
#### REDUCCION A K DE PRECIOS
###  EJM 5000 = 5K
df['price']=df['price']/1000

###ELIMIAR COLUMNAS QUE NO SIRVEN  DATOS COMO ID FECHA 

df=df.drop(['id', 'ad_type', 'start_date', 'end_date', 'created_on'],axis=1)


###ELIMIAR COLUMNAS QUE NO SIRVEN LUGAR COMO PAIS PERO MANTENIENDO DEPARTAMENTO

df=df.drop(['lat', 'lon', 'l1', 'l3', 'l4','l5','l6'],axis=1)

## ELIMINAR TITULO, MONEDA
df=df.drop([ 'description', 'title', 'operation_type'],axis=1)

df=df.drop([ 'currency',],axis=1)

#ELIMINAR PRICE PERIOD

df=df.drop([ 'price_period'],axis=1)

df=df.drop(['surface_total',
       'surface_covered'],axis=1)

####CAMBIAR ORDEN
df = df.reindex(columns=['l2', 'rooms', 'bedrooms', 'bathrooms',  'property_type', 'price'])

C:\Users\Hubble\AppData\Local\Temp\ipykernel_37144\4221343502.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price']=df['price']/1000


In [18]:
df


,l2,rooms,bedrooms,bathrooms,property_type,price
0,Lima,NaN,4.0,5.0,Casa,450.000
1,Arequipa,NaN,4.0,6.0,Casa,1050.000
2,La Libertad,NaN,5.0,4.0,Casa,179.000
3,Ica,1.0,NaN,NaN,Lote,52.000
4,Lima,1.0,NaN,NaN,Lote,70.000
...,...,...,...,...,...,...
100619,Arequipa,NaN,5.0,5.0,Casa,428.000
100620,Lima,NaN,6.0,5.0,Casa,699.000
100621,Lima,NaN,3.0,6.0,Casa,300.000
100622,Lima,NaN,4.0,6.0,Departamento,758.497


In [19]:
####DATAFRAME
X = df.iloc[:, :-1].values 
y = df.iloc[:,-1].values

In [20]:
##MEAN
print(df.rooms.mean())
print(df.bedrooms.mean())
print(df.bathrooms.mean())

3.669733210671573
2.8686621576245983
2.7089728994098325


In [21]:
##MEDIA 
import numpy as np 
print(np.median(df.rooms))
print(np.median(df.bedrooms))
print(np.median(df.bathrooms))


nan
nan
nan


In [22]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median') 

imputer = imputer.fit(X[:, 1:4])
X[:, 1:4] = imputer.transform(X[:,1:4])

pd.DataFrame(X)

,0,1,2,3,4
0,Lima,3.0,4.0,5.0,Casa
1,Arequipa,3.0,4.0,6.0,Casa
2,La Libertad,3.0,5.0,4.0,Casa
3,Ica,1.0,3.0,2.0,Lote
4,Lima,1.0,3.0,2.0,Lote
...,...,...,...,...,...
99962,Arequipa,3.0,5.0,5.0,Casa
99963,Lima,3.0,6.0,5.0,Casa
99964,Lima,3.0,3.0,6.0,Casa
99965,Lima,3.0,4.0,6.0,Departamento


In [23]:
##NOMBRES DE DEPARTAMENTOS EN NUMEROS 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])

pd.DataFrame(X)

,0,1,2,3,4
0,14,3.0,4.0,5.0,Casa
1,3,3.0,4.0,6.0,Casa
2,12,3.0,5.0,4.0,Casa
3,10,1.0,3.0,2.0,Lote
4,14,1.0,3.0,2.0,Lote
...,...,...,...,...,...
99962,3,3.0,5.0,5.0,Casa
99963,14,3.0,6.0,5.0,Casa
99964,14,3.0,3.0,6.0,Casa
99965,14,3.0,4.0,6.0,Departamento


In [24]:
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])

pd.DataFrame(X)

,0,1,2,3,4
0,14,3.0,4.0,5.0,0
1,3,3.0,4.0,6.0,0
2,12,3.0,5.0,4.0,0
3,10,1.0,3.0,2.0,6
4,14,1.0,3.0,2.0,6
...,...,...,...,...,...
99962,3,3.0,5.0,5.0,0
99963,14,3.0,6.0,5.0,0
99964,14,3.0,3.0,6.0,0
99965,14,3.0,4.0,6.0,3
